In [1]:
import random
import math
import time

# Define the parameters for data centers and tasks
edge_datacenters = [
    {
        'host_ram': 10,
        'num_hosts': 1,
        'storage': 0.5,
        'bandwidth': 5,
        'processing_power': 250000,
        'vm_number': 30,
        'accumulated_ram': 0.5,
        'accumulated_storage': 0.5,
        'cost': 0.022,
    }
]

cloud_datacenters = [
    {
        'host_ram': 20,
        'num_hosts': 1,
        'storage': 1,
        'bandwidth': 10,
        'processing_power': 1000000,
        'vm_number': 30,
        'accumulated_ram': 2,
        'accumulated_storage': 10,
        'cost': 0.051,
    }
]

num_tasks_range = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
random.seed(42)

# Simulated Annealing parameters
initial_temperature = 1000
cooling_rate = 0.95
iterations_per_temperature = 100

def initial_solution(num_tasks, num_datacenters):
    # Generate an initial random task allocation
    return [random.randint(0, num_datacenters - 1) for _ in range(num_tasks)]

def evaluate_solution(task_allocation, datacenters):
    # Calculate the resource utilization for the given task allocation
    total_utilization = 0
    total_cost = 0
    for task, datacenter_idx in enumerate(task_allocation):
        datacenter = datacenters[datacenter_idx]
        total_utilization += (datacenter['accumulated_ram'] + datacenter['accumulated_storage']) / (datacenter['host_ram'] + datacenter['storage'])
        total_cost += datacenter['cost']
    return total_utilization, total_cost

def neighbor_solution(current_solution, num_datacenters):
    # Generate a neighboring solution by moving a random task to a different data center
    neighbor = current_solution.copy()
    task_to_move = random.randint(0, len(neighbor) - 1)
    new_datacenter = random.randint(0, num_datacenters - 1)
    neighbor[task_to_move] = new_datacenter
    return neighbor

# Flower Pollination parameters
pollination_rate = 0.2

def flower_pollination(current_solution, num_datacenters):
    new_solution = current_solution.copy()

    num_perturb_tasks = int(pollination_rate * len(new_solution))

    for _ in range(num_perturb_tasks):
        task_to_perturb = random.randint(0, len(new_solution) - 1)
        new_datacenter = random.randint(0, num_datacenters - 1)
        new_solution[task_to_perturb] = new_datacenter

    return new_solution

# Simulated Annealing with Flower Pollination
def simulated_annealing_with_fp(num_tasks, datacenters):
    current_solution = initial_solution(num_tasks, len(datacenters))
    current_utilization, current_cost = evaluate_solution(current_solution, datacenters)
    best_solution = current_solution
    best_utilization, best_cost = current_utilization, current_cost
    temperature = initial_temperature

    start_time = time.time()

    while temperature > 1:
        for _ in range(iterations_per_temperature):
            perturbed_solution = flower_pollination(current_solution, len(datacenters))
            perturbed_utilization, perturbed_cost = evaluate_solution(perturbed_solution, datacenters)
            delta_utilization = perturbed_utilization - current_utilization

            if delta_utilization > 0 or random.random() < math.exp(delta_utilization / temperature):
                current_solution = perturbed_solution
                current_utilization = perturbed_utilization
                current_cost = perturbed_cost

                if current_utilization > best_utilization:
                    best_solution = current_solution
                    best_utilization = current_utilization
                    best_cost = current_cost

        temperature *= cooling_rate

    end_time = time.time()
    makespan = end_time - start_time

    return best_solution, best_utilization, best_cost, makespan

# Run the simulated annealing with flower pollination for edge and cloud data centers separately
def run_simulation(datacenters, datacenter_name):
    for num_tasks in num_tasks_range:
        total_makespan = 0
        total_cost = 0
        total_utilization = 0

        for _ in range(10):
            best_allocation, best_utilization, best_cost, makespan = simulated_annealing_with_fp(num_tasks, datacenters)
            total_utilization += best_utilization
            total_cost += best_cost
            total_makespan += makespan

        average_utilization = total_utilization / 10
        average_cost = total_cost / 10
        average_makespan = total_makespan / 10

        print(f"{datacenter_name} Data Center:")
        print(f"Number of tasks: {num_tasks}")
        print(f"Average Utilization: {average_utilization}")
        print(f"Average Cost: {average_cost}")
        print(f"Average Makespan: {average_makespan}\n")

# Run the simulation for both Edge and Cloud data centers
run_simulation(edge_datacenters, "Edge")
run_simulation(cloud_datacenters, "Cloud")

Edge Data Center:
Number of tasks: 100
Average Utilization: 9.523809523809506
Average Cost: 2.1999999999999993
Average Makespan: 0.7125029563903809

Edge Data Center:
Number of tasks: 200
Average Utilization: 19.047619047619
Average Cost: 4.399999999999986
Average Makespan: 1.310171866416931

Edge Data Center:
Number of tasks: 300
Average Utilization: 28.57142857142849
Average Cost: 6.60000000000001
Average Makespan: 2.05312066078186

Edge Data Center:
Number of tasks: 400
Average Utilization: 38.095238095237974
Average Cost: 8.800000000000036
Average Makespan: 2.841803789138794

Edge Data Center:
Number of tasks: 500
Average Utilization: 47.61904761904747
Average Cost: 11.000000000000059
Average Makespan: 3.520936679840088

Edge Data Center:
Number of tasks: 600
Average Utilization: 57.14285714285696
Average Cost: 13.200000000000085
Average Makespan: 4.80137312412262

Edge Data Center:
Number of tasks: 700
Average Utilization: 66.66666666666664
Average Cost: 15.400000000000109
Average

KeyboardInterrupt: 